In [5]:
import numpy as np
from generator import Croupier

Импортируем класс Croupier, который отвечает за генерацию данных и создадим экземпляр класса с заданными значениями вероятности для нечестного кубика, а так же вероятности перехода с честного кубика на нечестный и на себя самого, и вероятности перехода с нечестного кубика на себя и на честный. Проверим что получаем выпавшие знаечение кубика, а так же метку кубика

In [24]:
casino = Croupier([0.1,0.1,0.1,0.1,0.1,1/2],[0.05,0.95],[0.9,0.1])
for i in range(10):
    print(casino())

(1, 'F')
(6, 'F')
(2, 'U')
(3, 'U')
(1, 'U')
(2, 'U')
(4, 'U')
(2, 'F')
(3, 'F')
(1, 'F')


Сгенерируем датасеты размером 1 000, 10 000 и 100 000

In [29]:
dataset1000 = []
dataset1000Labels = []

for i in range(1000):
    data, label = casino()
    dataset1000.append(data)
    dataset1000Labels.append(label)

In [30]:
dataset10000 = []
dataset10000Labels = []

for i in range(10000):
    data, label = casino()
    dataset10000.append(data)
    dataset10000Labels.append(label)

In [31]:
dataset100000 = []
dataset100000Labels = []

for i in range(100000):
    data, label = casino()
    dataset100000.append(data)
    dataset100000Labels.append(label)

Определим базовый класс алгоритма

In [ ]:
class baseAlgo():
    def __init__(self, data, fairDiceStateProbability,
                 unfairDiceStateProbability, unfairDice, fairDice = None):
        self.data = data
        self.L = len(data)
        
        if fairDice == None: self.fairDice = [1/6 for i in range(6)]
        else: self.fairDice = fairDice
        self.unfairDice = unfairDice
        
        # Возможно стоит сделать вида 
        self.toFairState = np.array([fairDiceStateProbability[0],unfairDiceStateProbability[0]])
        self.toUnfariState = np.array([fairDiceStateProbability[1],unfairDiceStateProbability[1]])
        
        self.fairDiceStateProbability = fairDiceStateProbability
        self.unfairDiceStateProbability = unfairDiceStateProbability
        
        self.states = ['Fair','Unfair']
        self.dices = [self.fairDice,self.unfairDice]
        self.transitions = [self.toFairState, self.toUnfariState]
        
        self.path = [] #Вынести, так как прямой и обратной ход для другого

In [25]:
class algorithmViterbi(baseAlgo):
    def __init__(self, data, fairDiceStateProbability,
                 unfairDiceStateProbability, unfairDice, fairDice = None):
        baseAlgo.__init__(self, data, fairDiceStateProbability,
                 unfairDiceStateProbability, unfairDice, fairDice = None)
        self.v = [0 for i in range(self.L)]
        self.v[0] = 1
        
        self.pathProbabilities = []
        self.ptrStates = []
        
        
    def ptr(self,index, Fair2Current, Unfair2Current):
        return 'Fair' if self.pathProbabilities[index-1][0]*Fair2Current > self.pathProbabilities[index-1][1]*Unfair2Current else 'Unfar'
        
    def _stepRecursion(self, index, state, probabilityPerState):
        probabilityObservationsInState = probabilityPerState[data[i]-1]
        ###
        # Определить с помощью state и self.fairDiceStateProbability/self.unfairDiceStateProbability
        Fair2Current, Unfair2Current = 0,0 
        ###
        probability = probabilityPerState * (max(self.pathProbabilities[index-1][0]*Fair2Current,self.pathProbabilities[index-1][1]*Unfair2Current))
        argmax = self.ptr(index, Fair2Current, Unfair2Current)
        
        self.pathProbabilities.append(probability)
        self.ptrStates[-1].append(argmax)
    
    def __call__(self):
        # Рекурсия
        for i in range(1,self.L):
            self.pathProbabilitie.append([])
            self.ptrStates.append([])
            for state, probability in zip(['Fair','Unfair'],[self.fairDice, self.unfairDice]):
                self._stepRecursion(i,state,probability)
                
        # Завершение
        pass
        # Обратный ход
        pass

In [26]:
class forwardAlgorithm(baseAlgo):
    def __init__(self, data, fairDiceStateProbability,
                 unfairDiceStateProbability, unfairDice, fairDice = None):
        baseAlgo.__init__(self, data, fairDiceStateProbability,
                 unfairDiceStateProbability, unfairDice, fairDice = None)
        
        self.f = np.array([])
        self.pathProbability = None
    
    def step(self, index):
        self.f.append([])
        for transitionProbability, probability in zip(self.transitions, self.dices):
            observedProbability = probability[self.data[index]]
            previuosState = self.f[index-1]
            """
            observedProbability: вероятсность наблюдения при текущем состояние
            transitionProbability: вероятность перехода с кубика на кубик
            """
            probability = observedProbability * sum(previuosState * transitionProbability)
            self.f[-1].append(probability)
    
    def __call__(self):
        if self.pathProbability == None: self.initialization()
        return self.pathProbability
    
    def initialization(self):
        # Init
        self.f.append([1,0]) # Не уверен
        # Recursion
        for i in range(1,self.L):
            self.step(i)
        self.pathProbability = sum(self.f[-1])

In [27]:
class backwardAlgorithm(baseAlgo):
    def __init__(self, data, fairDiceStateProbability,
                 unfairDiceStateProbability, unfairDice, fairDice = None):
        baseAlgo.__init__(self, data, fairDiceStateProbability,
                 unfairDiceStateProbability, unfairDice, fairDice = None)
        pass
    
    def step(self):
        pass
    
    def __call__(self):
        pass
    
    def initialization(self):
        pass

In [28]:
class BaumWelchTraining():
    pass